In [12]:
# render ai gym environment
import gym
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
env = gym.make('MountainCarContinuous-v0')

#print(dir(env))

print('env.observation_space', env.observation_space)
#print(dir(env.observation_space))
print('env.observation_space.shape', env.observation_space.shape)
print('env.observation_space.low', env.observation_space.low)
print('env.observation_space.high', env.observation_space.high)

print('env.action_space', env.action_space)
#print(dir(env.action_space))
print('env.action_space.shape', env.action_space.shape)
print('env.action_space.low', env.action_space.low)
print('env.action_space.high', env.action_space.high)



print('env.reward_range', env.reward_range)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
env.observation_space Box(2,)
env.observation_space.shape (2,)
env.observation_space.low [-1.2  -0.07]
env.observation_space.high [0.6  0.07]
env.action_space Box(1,)
env.action_space.shape (1,)
env.action_space.low [-1.]
env.action_space.high [1.]
env.reward_range (-inf, inf)


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Policy(nn.Module):

    def __init__(self, state_size, action_size, action_low, action_high, phi_size, noise_std):
        super(Policy, self).__init__()
        
        self.state_size = state_size
        self.action_size = action_size
        self.action_low = np.expand_dims(action_low, 0)
        self.action_high = np.expand_dims(action_high, 0)
        self.phi_size = phi_size
        self.hidden_size = 64
        
        self.noise_std = noise_std

        self.fc1 = nn.Linear(self.state_size, self.hidden_size, bias=True)
        self.fc2 = nn.Linear(self.hidden_size, self.phi_size, bias=True)
        
        self.actor = nn.Linear(self.phi_size, self.action_size, bias=True)
        self.critic = nn.Linear(self.phi_size+self.action_size, 1, bias=True)
        
    def _phi(self, state):
        hidden = F.relu(self.fc1(state))
        phi = F.relu(self.fc2(hidden))
        return phi
    
    def _action(self, state):
        phi = self._phi(state)
        
        _actor_mean = F.sigmoid(self.actor(phi)) * (self.action_high - self.action_low) + self.action_low
        _actor_noise = torch.randn(_actor_mean.size()) * self.noise_std
        _actor_out = torch.clamp(_actor_mean + _actor_noise, self.action_low, self.action_high)
        
        return _actor_out, _actor_mean
    
    def _critic(self, state, action):
        _critic_out = self.critic(torch.cat([phi, action], dim=1))
        
        return _critic_out
       
    def forward(self, state, action):
    
        hidden = F.relu(self.fc1(state))
        phi = F.relu(self.fc2(hidden))
        
        _actor_mean = F.sigmoid(self.actor(phi)) * (self.action_high - self.action_low) + self.action_low
        _actor_noise = torch.randn(_actor_mean.size()) * self.noise_std
        _actor_out = torch.clamp(_actor_mean + _actor_noise, self.action_low, self.action_high)
        
        _critic_out = self.critic(phi)        

        return x
    
'''
    def forward(self, obs):
        phi = self.feature(obs)
        action = self.actor(phi)
        return action

    def feature(self, obs):
        obs = tensor(obs)
        return self.network.phi_body(obs)

    def actor(self, phi):
        return F.tanh(self.network.fc_action(self.network.actor_body(phi)))

    def critic(self, phi, a):
        return self.network.fc_critic(self.network.critic_body(phi, a))    '''

In [ ]:
# training loop max iterations
episode = 500

# widget bar to display progress
!pip install progressbar
import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episode).start()

# keep track of progress
mean_rewards = []

for e in range(episode):

    # collect trajectories
    old_probs, states, actions, rewards = \
        pong_utils.collect_trajectories(envs, policy, tmax=tmax)
        
    total_rewards = np.sum(rewards, axis=0)


    # gradient ascent step
    for _ in range(SGD_epoch):
        
        # uncomment to utilize your own clipped function!
        L = -clipped_surrogate(policy, old_probs, states, actions, rewards, epsilon=epsilon, beta=beta)

        # L = -pong_utils.clipped_surrogate(policy, old_probs, states, actions, rewards,
        #                                    epsilon=epsilon, beta=beta)
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        del L
    
    # the clipping parameter reduces as time goes on
    epsilon*=.999
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta*=.995
    
    # get the average reward of the parallel environments
    mean_rewards.append(np.mean(total_rewards))
    
    # display some progress every 20 iterations
    if (e+1)%20 ==0 :
        print("Episode: {0:d}, score: {1:f}".format(e+1,np.mean(total_rewards)))
        print(total_rewards)
        
    # update progress widget bar
    timer.update(e+1)
    
timer.finish()


In [ ]:
from parallelEnv import parallelEnv
import numpy as np
# keep track of how long training takes
# WARNING: running through all 800 episodes will take 30-45 minutes

# training loop max iterations
episode = 500

# widget bar to display progress
!pip install progressbar
import progressbar as pb
widget = ['training loop: ', pb.Percentage(), ' ', 
          pb.Bar(), ' ', pb.ETA() ]
timer = pb.ProgressBar(widgets=widget, maxval=episode).start()


envs = parallelEnv('PongDeterministic-v4', n=8, seed=1234)

discount_rate = .99
epsilon = 0.1
beta = .01
tmax = 320
SGD_epoch = 4

# keep track of progress
mean_rewards = []

for e in range(episode):

    # collect trajectories
    old_probs, states, actions, rewards = \
        pong_utils.collect_trajectories(envs, policy, tmax=tmax)
        
    total_rewards = np.sum(rewards, axis=0)


    # gradient ascent step
    for _ in range(SGD_epoch):
        
        # uncomment to utilize your own clipped function!
        L = -clipped_surrogate(policy, old_probs, states, actions, rewards, epsilon=epsilon, beta=beta)

        # L = -pong_utils.clipped_surrogate(policy, old_probs, states, actions, rewards,
        #                                    epsilon=epsilon, beta=beta)
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        del L
    
    # the clipping parameter reduces as time goes on
    epsilon*=.999
    
    # the regulation term also reduces
    # this reduces exploration in later runs
    beta*=.995
    
    # get the average reward of the parallel environments
    mean_rewards.append(np.mean(total_rewards))
    
    # display some progress every 20 iterations
    if (e+1)%20 ==0 :
        print("Episode: {0:d}, score: {1:f}".format(e+1,np.mean(total_rewards)))
        print(total_rewards)
        
    # update progress widget bar
    timer.update(e+1)
    
timer.finish()

In [ ]:
env.close()